In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, format_number

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Join Parquet") \
    .getOrCreate()


23/08/09 00:23:28 WARN Utils: Your hostname, Jasons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
23/08/09 00:23:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/09 00:23:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read the first Parquet file into a DataFrame
parquet_file_path1 = 'ol_works.snappy.parquet'
df1 = spark.read.parquet(parquet_file_path1)

In [3]:
# Read the second Parquet file into a DataFrame
parquet_file_path2 = 'ol_ratings.snappy.parquet'
df2 = spark.read.parquet(parquet_file_path2)

In [4]:
# Specify the join condition based on a common column (e.g., "work_key")
join_condition = ["work_key"]


In [5]:
# Perform the join operation
joined_df = df1.join(df2, on=join_condition, how="inner")
filtered_df = joined_df.filter(joined_df.rating >= 4)

In [6]:
grouped_df = joined_df.groupBy("title").agg(avg(col("rating")).alias("average_rating"))

In [7]:
grouped_df = grouped_df.withColumn("average_rating", format_number(col("average_rating"), 0))

In [8]:
grouped_df.show()

23/08/09 00:23:47 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+--------------+
|               title|average_rating|
+--------------------+--------------+
|       Karanlık Lise|             4|
|       Blood Promise|             4|
|           Red tears|             5|
|     Apresiasi puisi|             5|
|Bitcoin ATM Machi...|             5|
|    A Necessary Risk|             3|
|The Conviction of...|             4|
|          The Troika|             4|
|             Larceny|             5|
|El Miedo Escenico...|             3|
|The Juliette Society|             1|
|    American Ulysses|             5|
|Mrs. Peachtree an...|             5|
|    Midwinter Murder|             5|
| Play Fair with Love|             4|
|              Amulet|             5|
|         Shock Point|             5|
|Strong female pro...|             4|
|                 Kin|             5|
|          Riverworld|             4|
+--------------------+--------------+
only showing top 20 rows



In [11]:
grouped_df.repartition(1).write.csv("./data", header=True, mode="overwrite")

In [12]:
# grouped_df.repartition(1).write.parquet("./data", mode="overwrite")

23/08/09 00:01:36 ERROR Executor: Exception in task 1.0 in stage 7.0 (TID 20)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:71)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:374)
	at org.apache.parquet.bytes.HeapByteBufferAllocator.allocate(HeapByteBufferAllocator.java:32)
	at org.apache.parquet.hadoop.ParquetFileReader$ConsecutivePartList.readAll(ParquetFileReader.java:1773)
	at org.apache.parquet.hadoop.ParquetFileReader.internalReadRowGroup(ParquetFileReader.java:953)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextRowGroup(ParquetFileReader.java:909)
	at org.apache.parquet.hadoop.ParquetFileReader.readNextFilteredRowGroup(ParquetFileReader.java:1031)
	at org.apache.spark.sql.execution.datasources.parquet.SpecificParquetRecordReaderBase$ParquetRowGroupReaderImpl.readNextRowGroup(SpecificParquetRecordReaderBase.java:274)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordRea

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/ana

Py4JError: An error occurred while calling o62.parquet